# Sembradora 3000

This notebook presents an agent-based model that simulates the propagation of a disease through a network.
It demonstrates how to use the [agentpy](https://agentpy.readthedocs.io) package to create and visualize networks, use the interactive module, and perform different types of sensitivity analysis. 

In [1]:
# Model design
import agentpy as ap
import random
import numpy as np
from collections import namedtuple, deque
from queue import PriorityQueue
from itertools import count
import math

# Visualization
import matplotlib
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import matplotlib.image as mpimg
import matplotlib.animation as animation
import seaborn as sns
from IPython.display import HTML

## About the model

The agents of this model are people, which can be in one of the following three conditions: susceptible to the disease (S), infected (I), or recovered (R). The agents are connected to each other through a small-world network of peers. At every time-step, infected agents can infect their peers or recover from the disease based on random chance.

## Grid

In [ ]:
"""
1 is tractor
2 is obstacle
3 is target
4 is seeds
"""


def is_connected(grid, free_positions):
    """ Check if all free cells are connected using BFS """
    n = grid.shape[0]
    visited = set()
    queue = deque([free_positions.pop()])
    visited.add(queue[0])

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    
    connected_count = 1
    free_count = len(free_positions)

    while queue:
        x, y = queue.popleft()
        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < n and 0 <= new_y < n and (new_x, new_y) in free_positions and (new_x, new_y) not in visited:
                queue.append((new_x, new_y))
                visited.add((new_x, new_y))
                connected_count += 1
        
                
    return connected_count > free_count

def generate_grid(model, n, obstacles_count):
    grid = ap.Grid(model, (n, n), track_empty=True)  # Create an agentpy Grid object
    grid.add_field("occupied", 0)  # Add a field to store obstacle information

    # Generate obstacle positions
    obstacle_positions = set()
    while len(obstacle_positions) < obstacles_count:
        x, y = random.randint(0, n-1), random.randint(0, n-1)
        if (x, y) not in obstacle_positions and (x,y) not in model.p.seedsPositions:
            obstacle_positions.add((x, y))

    # Mark grid cells as obstacles
    for pos in obstacle_positions:
        grid["occupied"][pos] = 1

    # Identify free positions
    free_positions = set()
    for pos in grid.all:
        if grid["occupied"][pos] != 1:
            free_positions.add(pos)

    # Check if the free cells are connected
    final_obstacles = set()
    while not is_connected(grid, free_positions):
        final_obstacles.clear()
        grid = ap.Grid(model, (n, n), track_empty=True)
        grid.add_field("occupied", 0)
        obstacle_positions = set()
        while len(obstacle_positions) < obstacles_count:
            x, y = random.randint(0, n-1), random.randint(0, n-1)
            if (x, y) not in obstacle_positions and (x,y) not in model.p.seedsPositions:
                obstacle_positions.add((x, y))
        
        free_positions = set()
        for pos in grid.all:
            if grid["occupied"][pos] == 0:
                free_positions.add(pos)
        final_obstacles = obstacle_positions

    for pos in obstacle_positions:
        grid["occupied"][pos] = 1
        model.np_grid[pos] = 3
    #add an agent to each obstacle position with type 2
    #Make an agentlist ap.agentlist
    #Add the agent to the grid
    #Add the agent to the agentlist
    agentlist = ap.AgentList(model, len(obstacle_positions))
    agentlist.type = 2
        
    grid.add_agents(agentlist, obstacle_positions)
    


    model.grid = grid

## Agente

In [2]:
class CollectingTractor(ap.Agent):
    def setup(self, type=1, pos=(0, 0)):
        self.collected = 0
        self.targetIndex = 1
        self.path = []
        self.destroyed = False
        self.condition = True
        self.seeds = 0
        self.type = type
        
        # Machine Learning
        self.start = pos
        self.q_table = np.zeros((self.p.grid_size, self.p.grid_size, 6))  # 6 acciones posibles
        self.learning_rate = 0.1
        self.discount_factor = 0.9
        self.epsilon = 0.1
        self.pos = pos  # Posición inicial
        
        '''
        def heuristic(x1, y1, x2, y2):
        return abs(x1 - x2) + abs(y1 - y2)
    
    def a_star_search(self, start, end):
        #initialize the frontier using the initial state of the problem
        frontier = PriorityQueue()
        frontier.put(start, 0)
        came_from = {}
        cost_so_far = {}
        came_from[start] = None
        cost_so_far[start] = 0

        #run the search
        while not frontier.empty():
            current = frontier.get()

            if current == end:
                break

            #add to negihbors all the neighbors of the current cell
            neighbors = [(0, 1), (0, -1), (1, 0), (-1, 0)]
            size = self.model.grid.shape[0]

            for nextTemp in neighbors:
                #check if the next cell is within the grid
                next = (current[0]+ nextTemp[0], current[1] + nextTemp[1])
                if next[0] < 0 or next[0] >= size or next[1] < 0 or next[1] >= size:
                    continue
                #check if the next cell is an obstacle
                if self.model.grid["occupied"][next] == 1:
                    continue

                new_cost = cost_so_far[current] + 1
                if next not in cost_so_far or new_cost < cost_so_far[next]:
                    cost_so_far[next] = new_cost
                    priority = new_cost + abs(end[0] - next[0]) + abs(end[1] - next[1])
                    frontier.put(next, priority)
                    came_from[next] = current

        #reconstruct the path
        path = []
        current = end
        while current != start:
            path.append(current)
            current = came_from[current]
        path.reverse()
        return path
        
    def step(self):
        #run an A* search to find the shortest path to the target
        if (self.type == 2):
            return

        if (self.destroyed):
            return


        start = (self.pos[0], self.pos[1])
        if (self.seeds > 0):
            end = self.targets[self.targetIndex]
        else:
            #iterate through the seeds and find the closest one using A*
            minDist = 100000
            for seed in self.model.p.seedsPositions:
                dist = len(self.a_star_search(start, seed))
                if (dist < minDist):
                    minDist = dist
                    end = seed

        self.model.np_grid[end] = 2
        if (len(self.path) == 0):
            self.path = self.a_star_search(start, end)

        
        if (len(self.path) == 0):
            next_pos = self.pos
        else:
            next_pos = self.path.pop(0)
            if not (next_pos in self.model.grid.empty.items):
                return        
        
        self.model.grid.move_to(self, next_pos)
        self.model.np_grid[self.pos] = 0
        self.model.np_grid[next_pos] = 1
        self.pos = next_pos

        if (self.pos in self.model.p.seedsPositions):
            self.seeds = self.capacity
            return
        #check if the target has been reached

        if self.pos == end:
            self.seeds -= 1
            self.model.Collected += 1
            self.targetIndex += 1
            if self.targetIndex == len(self.targets):
                self.model.np_grid[self.pos] = 0
                self.model.grid.remove_agents(self)
                self.destroyed = True
                self.condition = False
                self.type = 0
        '''
        
    # Función para moverse arrriba
    def move_up(self):
        x, y = self.pos
        if y < self.p.grid_size - 1:
            self.model.grid.move_to(self, (x, y + 1))
            self.pos = (x, y + 1)
            return self.pos, True
        else:
            return self.pos, False
        
    # Función para moverse abajo
    def move_down(self):
        x, y = self.pos
        if y > 0:
            self.model.grid.move_to(self, (x, y - 1))
            self.pos = (x, y - 1)
            return self.pos, True
        else:
            return self.pos, False
        
    # Función para moverse a la izquierda
    def move_left(self):
        x, y = self.pos
        if x > 0:
            self.model.grid.move_to(self, (x - 1, y))
            self.pos = (x - 1, y)
            return self.pos, True
        else:
            return self.pos, False
        
    # Función para moverse a la derecha
    def move_right(self):
        x, y = self.pos
        if x < self.p.grid_size - 1:
            self.model.grid.move_to(self, (x + 1, y))
            self.pos = (x + 1, y)
            return self.pos, True
        else:
            return self.pos, False
    
    # función para dejar una semilla
    def drop(self):
        x, y = self.pos
        if self.seeds > 0:
            self.seeds -= 1
            self.model.grid["occupied"][self.pos] = 4
            self.model.np_grid[self.pos] = 4
            return self.pos, True
        else:
            return self.pos, False
            
    # función para recoger una semillas
    '''Este esta mal, no se como hacerlo'''
    def collect(self):
        x, y = self.pos
        if self.model.grid["occupied"][self.pos] == 4: # If semillas == 0, agarra semillas
            self.seeds += 1
            self.model.grid["occupied"][self.pos] = 0
            self.model.np_grid[self.pos] = 0
            return self.pos, 1
        elif self.seeds < self.capacity: # If semillas > 0 y semillas menor a capacidad maxima, agarra semillas
            self.seeds += 1
            self.model.grid["occupied"][self.pos] = 0
            self.model.np_grid[self.pos] = 0
            return self.pos, 2
        else: # Si semillas == capacidad maxima, no agarra semillas
            return self.pos, 3
    
    
    def get_reward(self, action, agents, estado):
        if action == 0:  # Arriba
            if estado == True:
                reward = -5  # Penalización por step
            else:
                reward = -15  # Penalización por intentar salir del límite
        elif action == 1:  # Abajo
            if estado == True:
                reward = -5
            else:
                reward = -15
        elif action == 2:  # Izquierda
            if estado == True:
                reward = -5
            else:
                reward = -15
        elif action == 3:  # Derecha
            if estado == True:
                reward = -5
            else:
                reward = -15
        elif action == 4:  # Dejar
            if estado == True:
                reward = 10  # Recompensa positiva por dejar semillas
            else:
                reward = -15  # Recompensa negativa por intentar dejar sin semillas
        elif action == 5:  # Recoger
            if estado == 1:
                reward = -10
            elif estado == 2:
                reward = -5
            else:
                reward = -15
        else:
            # Penalización pequeña por cada movimiento para incentivar la eficiencia
            reward = -10

        # Verificación de colisiones con otros agentes
        for agent in agents:
            if agent != self and agent.pos == self.pos:
                reward -= 5  # Penalización por colisión con otro agente
                break

        return reward

    def q_learning_update(self, state, action, reward, next_state):
        current_q = self.q_table[state][action]
        max_next_q = np.max(self.q_table[next_state])
        new_q = current_q + self.learning_rate * (reward + self.discount_factor * max_next_q - current_q)
        self.q_table[state][action] = new_q

    def acciones(self, action):
        x, y = self.pos

        if action == 0:  # Arriba
            new_pos, accionado  = (x, y + 1) if y < self.p.grid_size - 1 else self.pos
        elif action == 1:  # Abajo
            new_pos, accionado = (x, y - 1) if y > 0 else self.pos
        elif action == 2:  # Izquierda
            new_pos = (x - 1, y) if x > 0 else self.pos
        elif action == 3:  # Derecha
            new_pos = (x + 1, y) if x < self.p.grid_size - 1 else self.pos
        elif action == 4:  # Dejar
            self.drop()
            pass
        elif action == 5:  # Recoger
            self.collect()
            pass

        return new_pos, accionado

    def step(self, agents):
        state = self.pos
        if np.random.rand() < self.epsilon:
            action = np.random.choice([0, 1, 2, 3, 4, 5])  # Incluyendo acciones de dejar y recoger
        else:
            action = np.argmax(self.q_table[state])

        next_state, estado = self.acciones(action)
        reward = self.get_reward(action, agents, estado)
        self.q_learning_update(state, action, reward, next_state)

        self.pos = next_state  # Actualizar la posición del agente


#### Ambiente

In [4]:
class TractorModel(ap.Model):
    def setup(self):
        self.Collected = 0
        #Numpy array of size grid_size x grid_size
        self.np_grid = np.zeros((self.p.grid_size, self.p.grid_size))
        generate_grid(self, self.p.grid_size, self.p.obstacles_count)
        #Create p.number_of_tractors tractors
        self.agents = ap.AgentList(self, self.p.number_of_tractors, CollectingTractor)
        self.agents.capacity = self.p.capacity
        self.agents.seeds = self.p.starting_seeds
        #Unique coords
        coordsUsed = set()
        #Set the targets for each tractor, checking that they are not obstacles
        for tractor in self.agents:
            targets = []
            for i in range(self.p.number_of_targets + 1):
                x, y = random.randint(0, self.p.grid_size-1), random.randint(0, self.p.grid_size-1)
                while self.grid["occupied"][(x, y)] == 1 or (x, y) in coordsUsed:
                    x, y = random.randint(0, self.p.grid_size-1), random.randint(0, self.p.grid_size-1)
                targets.append((x, y))
                coordsUsed.add((x, y))
            tractor.targets = targets
            tractor.pos = targets[0]
            self.np_grid[tractor.pos] = 1
        
        self.grid.add_agents(self.agents, [tractor.pos for tractor in self.agents])
        
    def update(self):
        self.record('Collected', sum([tractor.collected for tractor in self.agents]))

        
    def step(self):
        self.agents.step()
        #Assign 4 to the seed positions
        for seed in self.p.seedsPositions:
            self.np_grid[seed] = 4
        
    def end(self):
        self.report('Total targets', self.p.number_of_targets * self.p.number_of_tractors)
        #time to collect all targets

## Parameters

In [5]:


tractorParameters = {
    'grid_size': 10,
    'obstacles_count': 10,
    'number_of_tractors': 4,
    'number_of_targets': 8,
    'steps': 100,
    'seedsPositions': [(0, 0)],
    'capacity': 2,
    'starting_seeds': 2
}
model = TractorModel(tractorParameters)
results = model.run(steps=100)
'''model = VirusModel(parameters)
results = model.run() '''

Completed: 100 steps
Run time: 0:00:00.021634
Simulation finished


'model = VirusModel(parameters)\nresults = model.run() '

## Machine Learning

## Analyzing results

In [ ]:
''' 
ESTO NO JALA, PERO SI ALGUIEN LO QUIERE ARREGLAR, DESE
def tractor_plot(data, ax):
    x = data.index.get_level_values('t')
    y = data['Collected']
    ax.plot(x, y, label='Collected targets')
    ax.legend()
    ax.set_xlim(0, max(1, len(x)-1))
    ax.set_ylim(0, 25)
    ax.set_xlabel("Time steps")
    ax.set_ylabel("Number of collected targets")

fig, ax = plt.subplots()
tractor_plot(results.variables.TractorModel, ax)
'''

## Creating an animation

In [ ]:
"""
0 is empty
1 is tractor
2 is obstacle
3 is target
4 is seeds
"""

# Load images
tractor_img = mpimg.imread('tractor.png')
obstacle_img = mpimg.imread('obstacle.png')
target_img = mpimg.imread('target.png')
seeds_img = mpimg.imread('seeds.png')

def animation_plot(model, ax):
    # Clear the axis to avoid over-plotting
    ax.clear()

    # Plot the grid using images
    for (x, y), value in np.ndenumerate(model.np_grid):
        if value == 1:  # Tractor
            ax.imshow(tractor_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 3:  # Obstacle
            ax.imshow(obstacle_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 2:  # Target
            ax.imshow(target_img, extent=[y, y+1, x, x+1], aspect='auto')
        elif value == 4:  # Seeds
            ax.imshow(seeds_img, extent=[y, y+1, x, x+1], aspect='auto')

    # Add text for each tractor displaying the number of seeds it has
    for agent in model.agents:
        if agent.destroyed:
            continue
        ax.text(agent.pos[1] + 0.5, agent.pos[0] + 0.5, str(agent.seeds),
                color='black', fontsize=12, ha='center', va='center', weight='bold')

    # Fix axis limits based on the grid dimensions
    ax.set_xlim([0, model.np_grid.shape[1]])
    ax.set_ylim([0, model.np_grid.shape[0]])

    # Set aspect ratio to 'equal' to prevent image stretching
    ax.set_aspect('equal')

    # Set the title for the plot
    ax.set_title(f"Tractor model \n Time-step: {model.t}, Collected: {model.Collected}")

# Example usage
fig, ax = plt.subplots()
model = TractorModel(tractorParameters)
animation = ap.animate(model, fig, ax, animation_plot)
animation.save('simulacionTractores.gif')


In [ ]:
#Display the animation
HTML(animation.to_html5_video())